# Redpoll vs machine learning

In [1]:
with open('paths.txt') as f:
    paths = {}
    for line in f.readlines():
        filename, path = line.split('=')
        paths[filename.strip()] = path.strip()
        
path = paths['satellites']

# Data Preparation and Training

In [2]:
import pandas as pd

df = pd.read_csv(path, index_col=0)
df.shape

(1164, 20)

In [3]:
continuous_columns = [
    'Perigee_km', 'Apogee_km', 'Eccentricity',
    'Period_minutes', 'Launch_Mass_kg',
    'Dry_Mass_kg', 'Power_watts',
    'Date_of_Launch', 'Expected_Lifetime',
    'longitude_radians_of_geo',
    'Inclination_radians'
]

target_column = "Expected_Lifetime"

for col in list(df.columns):
    # don't mess w/ the target
    if col == target_column:
        continue
        
    if col in continuous_columns:
        # fill in missing values with mean
        df[col].fillna(df[col].median(), inplace=True)
        
    else:
        # fill in missing values with mode
        df[col].fillna(df[col].mode()[0], inplace=True)
        
        # convert strings to categories
        df[col] = df[col].astype('category').cat.codes

        # create and lable one-hot features
        one_hot = pd.get_dummies(df[col])
        one_hot.columns = ["%s_%d" % (col, label) for
                           label in one_hot]
        
        # replace categorical column w/ one-hot dummies
        df.drop([col], axis=1)
        df = pd.concat([df, one_hot], axis=1)

In [4]:
df_el_in = df.loc[~df[target_column].isnull(), :]
df_el_out = df.loc[df[target_column].isnull(), :]
df_el_in.shape

(878, 432)

In [5]:
from sklearn.ensemble import RandomForestRegressor


el_predictor_columns = [col for col in df_el_in.columns if
                        col != target_column]
el_x = df_el_in.drop(target_column, axis=1)
el_y = df_el_in[target_column]

rf_el = RandomForestRegressor()
rf_el.fit(el_x, el_y)

RandomForestRegressor()

In [6]:
# DO NOT INCLUDE IN DEMO OUTPUT


# Parameter Selection, Feature Selection, and Validation

In [7]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_validate

In [8]:
%%time
import itertools as it
 
max_features = ['auto', 'sqrt', 'log2']
criterion = ['mse', 'mae']
n_estimators = [10, 100, 500]
 
best_score = None
best_params = None
for params in it.product(max_features, criterion, n_estimators):
    ftrs, crit, n_est = params
    
    estimator = RandomForestRegressor(
        max_features=ftrs,
        criterion=crit,
        n_estimators=n_est
    )
    
    scores = cross_validate(
      estimator,
      el_x,
      el_y,
      scoring=('r2',)
    )
    score = sum(scores["test_r2"])

    if best_score is None:
        best_score = score
        best_params = params
        
    if score > best_score:
        best_score = score
        best_params = params

CPU times: user 9min 55s, sys: 709 ms, total: 9min 55s
Wall time: 9min 56s


In [9]:
best_params

('log2', 'mse', 500)

In [10]:
best_score

2.7843662111816583

In [11]:
el_model = RandomForestRegressor(
    max_features='sqrt',
    criterion='mae',
    n_estimators=100
)
el_model.fit(el_x, el_y)

el_importance = pd.DataFrame({
    "Feature": el_x.columns,
    "Importance": el_model.feature_importances_
})

el_importance = el_importance\
    .sort_values(by=['Importance'], ascending=False)

el_features = el_importance.Feature.head(20).values
el_x2 = el_x[el_features]

scores = cross_validate(
    el_model,
    el_x2,
    el_y,
    scoring=('r2',)
)
score = sum(scores["test_r2"])
score

2.5767812037034776

# Prediction

In [12]:
predictors = [col for col in df_el_out.columns if
              col != target_column]
el_predictors = df_el_out.drop(target_column, axis=1)
el_predictions = el_model.predict(el_predictors)

# Classification

In [13]:
df = pd.read_csv(path, index_col=0)
target_column = "Type_of_Orbit"
 
for col in list(df.columns):
    # don't mess w/ the target
    if col == target_column:
        continue
        
    if col in continuous_columns:
        # fill in missing values with mean
        df[col].fillna(df[col].median(), inplace=True)
        
    else:
        # fill in missing values with mode
        df[col].fillna(df[col].mode()[0], inplace=True)
        
        # convert string categories to int codes
        df[col] = df[col].astype('category').cat.codes
        
        # create and lable one-hot features
        one_hot = pd.get_dummies(df[col])
        one_hot.columns = ["%s_%d" % (col, label) for
                           label in one_hot]
        
        # replace categorical column w/ one-hot dummies
        df.drop([col], axis=1)
        df = pd.concat([df, one_hot], axis=1)

df_co_in = df.loc[~df[target_column].isnull(), :]
df_co_out = df.loc[df[target_column].isnull(), :]

In [14]:
df_co_in.shape

(519, 425)

In [15]:
df_co_out.shape

(645, 425)

In [16]:
%%time
co_x = df_co_in[[col for col in df_co_in.columns if
                 col != target_column]]
co_y = df_co_in[target_column]

from sklearn.ensemble import RandomForestClassifier

max_features = ['auto', 'sqrt', 'log2']
criterion = ['gini', 'entropy']
n_estimators = [10, 100, 500]

best_score = None
best_params = None
all_scores = []
for params in it.product(max_features, criterion,
                         n_estimators):
    ftrs, crit, n_est = params
    
    estimator = RandomForestClassifier(
        max_features=ftrs,
        criterion=crit,
        n_estimators=n_est,
    )
    
    scores = cross_validate(estimator, co_x, co_y,
                            scoring=('f1_micro',))
    score = sum(scores["test_f1_micro"])
    all_scores.append(score)
    if best_score is None:
        best_score = score
        best_params = params
        
    if score > best_score:
        best_score = score
        best_params = params

/Users/bax/.local/share/virtualenvs/demos-cxGizJPl/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
/Users/bax/.local/share/virtualenvs/demos-cxGizJPl/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
/Users/bax/.local/share/virtualenvs/demos-cxGizJPl/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
/Users/bax/.local/share/virtualenvs/demos-cxGizJPl/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 me

CPU times: user 25 s, sys: 95.5 ms, total: 25.1 s
Wall time: 25.2 s


In [17]:
(best_score, best_params)

(4.48777072442121, ('auto', 'gini', 10))

In [18]:
co_model = RandomForestClassifier(
    max_features='auto',
    criterion='entropy',
    n_estimators=10
)
co_model.fit(co_x, co_y)
co_importance = pd.DataFrame({
    "Feature": co_x.columns,
    "Importance": co_model.feature_importances_
})
co_importance = co_importance\
    .sort_values(by=['Importance'], ascending=False)
co_features = co_importance.Feature.head(20).values
co_x2 = co_x[co_features]
scores = cross_validate(co_model, co_x2, co_y,
                        scoring=('f1_micro',))
score = sum(scores["test_f1_micro"])
score

/Users/bax/.local/share/virtualenvs/demos-cxGizJPl/lib/python3.8/site-packages/sklearn/model_selection/_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


4.5661874533233755

# Prediction Certainty

In [19]:
import forestci
forestci.random_forest_error(
    rf_el,
    el_x,
    el_predictors,
)       

Failed to import duecredit due to No module named 'duecredit'
/Users/bax/.local/share/virtualenvs/demos-cxGizJPl/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/bax/.local/share/virtualenvs/demos-cxGizJPl/lib/python3.8/site-packages/forestci/calibration.py:86: RuntimeWarning: overflow encountered in exp
  g_eta_raw = np.exp(np.dot(XX, eta)) * mask
/Users/bax/.local/share/virtualenvs/demos-cxGizJPl/lib/python3.8/site-packages/forestci/calibration.py:86: RuntimeWarning: overflow encountered in exp
  g_eta_raw = np.exp(np.dot(XX, eta)) * mask


array([25.29567825, 15.60084148, 14.92409254, 24.46440549, 21.49261344,
       25.8735099 , 15.65533839, 15.65533839, 15.64922336, 14.46928408,
       13.97224021, 15.03269183, 15.29980005, 14.34378368, 14.79864907,
       14.52204182, 14.57212478, 15.22525075, 14.97925583, 18.29398241,
       14.96060358, 28.91331392, 13.679016  , 15.65433821, 16.44282146,
       14.61110128, 14.54397613, 14.86451187, 14.5338824 , 15.13956617,
       12.92732877, 14.92319685, 14.98319532, 14.83430341, 15.70166742,
       15.9478725 , 15.92192739, 15.56621343, 12.79198131, 13.04866396,
       13.97631056, 18.95748385, 16.49204568, 16.46824913, 17.29373359,
       16.90372166, 15.72753762, 15.34009276, 16.71391435, 22.73978001,
       17.10758019, 17.10758019, 17.147215  , 17.5897531 , 15.08673539,
       15.07967617, 15.08673539, 15.07967617, 23.16792473, 20.53514385,
       15.10024843, 16.47294393, 15.60854149, 15.27227335, 15.27227335,
       14.94458393, 14.83690817, 14.66464055, 16.01166842, 13.33

# Anomaly detection

In [20]:
pm = df.Period_minutes[~df.Period_minutes.isnull()]
pm_dist = ((pm-pm.mean())/pm.std()).abs()
outliers = pm_dist[pm_dist > 3]
pm = df.Period_minutes.loc[list(outliers.index)]\
    .sort_values(ascending=False)
pm.head()

ID
Wind (International Solar-Terrestrial Program)                19700.45
Integral (INTErnational Gamma-Ray Astrophysics Laboratory)     4032.86
Chandra X-Ray Observatory (CXO)                                3808.92
Tango (part of Cluster quartet, Cluster 2 FM8)                 3442.00
Rumba (part of Cluster quartet, Cluster 2 FM5)                 3431.10
Name: Period_minutes, dtype: float64